# nb-model_xg-model-neural-process-0

In [1]:
import sys
import os
from os import sep
from os.path import dirname, realpath
from pathlib import Path
from collections import OrderedDict
from functools import partial, reduce
import logging

def get_cwd(fname, subdir, crunch_dir=realpath(Path.home()) +sep +'crunch' +sep):
    """
    Convenience function to make a directory string for the current file based on inputs.
    Jupyter Notebook in Anaconda invokes the Python interpreter in Anaconda's subdirectory
    which is why changing sys.argv[0] is necessary. In the future a better way to do this
    should be preferred..
    """
    return crunch_dir +subdir +fname

def fix_path(cwd):
    """
    Convenience function to fix argv and python path so that jupyter notebook can run the same as
    any script in crunch.
    """
    sys.argv[0] = cwd
    module_path = os.path.abspath(os.path.join('..'))
    if module_path not in sys.path:
        sys.path.append(module_path)

fname = 'nb-model_xg-model-neural-process.ipynb'
dir_name = 'model'
fix_path(get_cwd(fname, dir_name +sep))

import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
from dask import delayed, compute
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.dataset import Dataset as TorchDataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import weight_norm
import torchfunc
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping

from ipywidgets import interact, interactive, fixed
from IPython.display import display

pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 50)

from common_util import MODEL_DIR, RECON_DIR, JSON_SFX_LEN, DT_CAL_DAILY_FREQ, is_valid, pd_to_np, pairwise, df_midx_restack, compose, is_type, df_rows_in_year, list_all_eq, remove_dups_list, NestedDefaultDict, set_loglevel, search_df, chained_filter, get_variants, load_df, dump_df, load_json, gb_transpose, pd_common_index_rows, filter_cols_below, inner_join, outer_join, ser_shift, list_get_dict, window_iter, benchmark
from common_util import pd_rows_key_in, df_add_midx_level, df_del_midx_level, pd_split_ternary_to_binary, np_value_counts, isnt, window_iter, all_eq, np_assert_identical_len_dim, pd_idx_rename, midx_get_level, pd_rows, midx_intersect, pd_get_midx_level, pd_common_idx_rows, midx_split, pd_midx_to_arr, window_iter, np_at_least_nd, np_is_ndim, identity_fn
from model.common import DATASET_DIR, XG_PROCESS_DIR, XG_DATA_DIR, XG_DIR, PYTORCH_MODELS_DIR, TEST_RATIO, VAL_RATIO, EXPECTED_NUM_HOURS, default_dataset
from model.common import PYTORCH_ACT_MAPPING, PYTORCH_OPT_MAPPING, PYTORCH_SCH_MAPPING, PYTORCH_LOSS_MAPPING
from model.xg_util import xgload, get_xg_feature_dfs, get_xg_label_target_dfs
from model.train_util import pd_to_np_tvt, temporal_preproc_3d, stride_preproc_3d, batchify, get_dataloader
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

CRITICAL:root:script location: /home/kev/crunch/model/nb-model_xg-model-fr-model.ipynb
CRITICAL:root:using project dir: /home/kev/crunch/


Prune the xg data down to the data of interest to use in further experiments.

## Load Data

In [2]:
assets = ['sp_500', 'russell_2000', 'nasdaq_100', 'dow_jones']
chosen_asset = assets[0]

/home/kev/miniconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
fd = get_xg_feature_dfs(chosen_asset)
ld, td = get_xg_label_target_dfs(chosen_asset)

In [4]:
d_all_common = ['dwrmx', 'dwrod', 'dwrpt', 'dwrzn', 'dffd']
d_extra = ['dlogret', 'dohlca', 'ddiff', 'dc', 'dwrxmx']

/home/kev/miniconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
h_all_common = ['hdmx', 'hdod', 'hdpt', 'hdzn']
h_extra = ['hdgau', 'hduni', 'hohlca']

In [6]:
h_all_common, d_all_common

(['hdmx', 'hdod', 'hdpt', 'hdzn'],
 ['dwrmx', 'dwrod', 'dwrpt', 'dwrzn', 'dffd'])

## Select Data

### Choose Features and Join

In [7]:
## Print all the available features:
# for k0, v0 in fd.items():
#     print(k0)
#     for k1,v1 in fd[k0].items():
#         print('\t' +k1)
#         for k2, v2 in fd[k0][k1].items():
#             print('\t\t' +k2)
#             for k3, v3 in fd[k0][k1][k2].items():
#                 print('\t\t\t' +k3)

In [8]:
#for ser_name in chosen_f.index.unique(level='id1'):
#    print('\t' +ser_name)
#    ser_subset_df = chosen_f.xs(ser_name, level=1, drop_level=False)
#    ser_subset_df.index = ser_subset_df.index.remove_unused_levels()
#    display(pd_to_np(ser_subset_df))
#    ser_subset_df.isnan()
#    np.isnan(ser_subset_df.values).any()

In [9]:
from xg_util import get_hardcoded_daily_dfs, get_hardcoded_hourly_dfs

In [10]:
def df_filter_by_keywords(fdf, keywords):
	cols = [col for col in fdf.index.levels[1] if (any(keyword in col for keyword in keywords))]
	return pd_rows_key_in(fdf, 'id1', cols)

def get_hardcoded_daily_dfs(fd, src):
	"""
	Return hardcoded daily freq feature dfs to use in experiments.
	"""
	feature_dfs = []

	#if (src in ('pba', 'vol')):
	#	feature_dfs.append(fd['d'][src]['dlogret']['{src}_dlh_dlogret'.format(src=src)])

	if (src in ('pba', 'vol', 'buzz')):
		for axe in ('dwrmx', 'dwrod', 'dwrpt', 'dwrzn', 'dffd'):
		#for axe in ('dwrmx', 'dwrod', 'dwrpt', 'dwrzn'):
			for fdf in fd['d'][src][axe].values():
				#feature_dfs.append(df_filter_by_keywords(fdf, ('avgPrice', 'high', 'low')))
				#feature_dfs.append(df_filter_by_keywords(fdf, ('avgPrice', 'high', 'low', 'close')))
				feature_dfs.append(fdf)
				#feature_dfs.append(df_filter_by_keywords(df_filter_by_keywords(fdf, ('avgPrice', 'high', 'low')), ('(5)', '(20)',)))

	if (src in ('nonbuzz',)):
		for axe in ('dffd', 'dwrxmx'):
			for fdf in fd['d'][src][axe].values():
				feature_dfs.append(fdf)

	#return pd.concat(feature_dfs).sort_index(axis=0)
	return pd.concat(feature_dfs)

def get_hardcoded_hourly_dfs(fd, src):
	"""
	Return hardcoded hourly freq feature dfs to use in experiments.
	"""
	feature_dfs = []

	if (src in ('pba', 'vol', 'buzz')):
		for axe in ('hdmx', 'hdod', 'hdpt', 'hdzn', 'hdgau', 'hduni'):
			for fdf in fd['h'][src][axe].values():
				#feature_dfs.append(df_filter_by_keywords(fdf, ('avgPrice', 'high', 'low', 'close')))
				feature_dfs.append(fdf)

	return pd.concat(feature_dfs)

In [11]:
d_pba = get_hardcoded_daily_dfs(fd, 'pba')
d_vol = get_hardcoded_daily_dfs(fd, 'vol')
d_buzz = get_hardcoded_daily_dfs(fd, 'buzz')
d_nonbuzz = get_hardcoded_daily_dfs(fd, 'buzz')
h_pba = get_hardcoded_hourly_dfs(fd, 'pba')
h_vol = get_hardcoded_hourly_dfs(fd, 'vol')
h_buzz = get_hardcoded_hourly_dfs(fd, 'buzz')

In [12]:
# Returns (AR)
chosen_f = fd['d']['pba']['dret']['pba_hlh_hdxret_dret']
chosen_f = fd['d']['pba']['dret']['pba_hoc_hdxret_dret']
#chosen_f = fd['d']['pba']['ddir']['pba_hoc_hdxret_ddir']

In [13]:
# Daily Features
chosen_f = d_pba
chosen_f = fd['d']['pba']['ddir']['pba_hoc_hdxret_ddir']

In [14]:
fd['d']['pba']

{'ddiff': {'pba_dohlca_ddiff(1)':                                                     -1
  id0        id1                                        
  1996-01-03 pba_open_pba_dohlca_ddiff(1)       4.800000
             pba_high_pba_dohlca_ddiff(1)       2.510000
             pba_low_pba_dohlca_ddiff(1)        6.390000
             pba_close_pba_dohlca_ddiff(1)      0.590000
             pba_avgPrice_pba_dohlca_ddiff(1)   3.976738
  ...                                                ...
  2018-01-08 pba_open_pba_dohlca_ddiff(1)      11.340000
             pba_high_pba_dohlca_ddiff(1)       5.060000
             pba_low_pba_dohlca_ddiff(1)        9.680000
             pba_close_pba_dohlca_ddiff(1)      4.560000
             pba_avgPrice_pba_dohlca_ddiff(1)   9.365525
  
  [27550 rows x 1 columns]},
 'ddir': {'pba_hlh_hdxret_ddir':                                         -1
  id0        id1                            
  1996-01-02 pba_lh_pba_hlh_hdxret_ddir  1.0
  1996-01-03 pba_lh_pba_hlh_h

In [15]:
fd['d']['pba']['dohlca']['pba_dohlca'].unstack().pct_change().dropna().corr()

/home/kev/miniconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


-1                      \
id1                        pba_open_pba_dohlca pba_high_pba_dohlca   
   id1                                                               
-1 pba_open_pba_dohlca                1.000000            0.044413   
   pba_high_pba_dohlca                0.044413            1.000000   
   pba_low_pba_dohlca                 0.176343            0.013205   
   pba_close_pba_dohlca               0.003364            0.010959   
   pba_avgPrice_pba_dohlca            0.481159            0.065632   

                                                                    \
id1                        pba_low_pba_dohlca pba_close_pba_dohlca   
   id1                                                               
-1 pba_open_pba_dohlca               0.176343             0.003364   
   pba_high_pba_dohlca               0.013205             0.010959   
   pba_low_pba_dohlca                1.000000             0.963043   
   pba_close_pba_dohlca              0.963043             1.000000   
   pba_avgPrice_pba_dohlca           0.273103             0.244556   

                                                    
id1                        pba_avgPrice_pba_dohlca  
   id1                                              
-1 pba_open_pba_dohlca                    0.481159  
   pba_high_pba_dohlca                    0.065632  
   pba_low_pba_dohlca                     0.273103  
   pba_close_pba_dohlca                   0.244556  
   pba_avgPrice_pba_dohlca                1.000000

In [16]:
fd['d']['vol']['dohlca']['vol_dohlca'].unstack().pct_change().dropna().corr()

/home/kev/miniconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


-1                      \
id1                        vol_open_vol_dohlca vol_high_vol_dohlca   
   id1                                                               
-1 vol_open_vol_dohlca                1.000000            0.483537   
   vol_high_vol_dohlca                0.483537            1.000000   
   vol_low_vol_dohlca                 0.662008            0.164855   
   vol_close_vol_dohlca               0.129246            0.414403   
   vol_avgPrice_vol_dohlca            0.375900            0.502796   

                                                                    \
id1                        vol_low_vol_dohlca vol_close_vol_dohlca   
   id1                                                               
-1 vol_open_vol_dohlca               0.662008             0.129246   
   vol_high_vol_dohlca               0.164855             0.414403   
   vol_low_vol_dohlca                1.000000             0.308431   
   vol_close_vol_dohlca              0.308431             1.000000   
   vol_avgPrice_vol_dohlca           0.436877             0.736014   

                                                    
id1                        vol_avgPrice_vol_dohlca  
   id1                                              
-1 vol_open_vol_dohlca                    0.375900  
   vol_high_vol_dohlca                    0.502796  
   vol_low_vol_dohlca                     0.436877  
   vol_close_vol_dohlca                   0.736014  
   vol_avgPrice_vol_dohlca                1.000000

In [17]:
len(h_buzz.index.levels[1])

/home/kev/miniconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


96

In [18]:
# Hourly Features
chosen_f = h_pba
chosen_f = fd['h']['pba']['hdmx']['pba_hohlca_hdmx']

In [19]:
# # Random 'Features':
# chosen_f = pd.DataFrame(np.random.rand(len(chosen_f)), index=chosen_f.index).rename({chosen_f.index.levels[1][0]: 'rand'})

### Choose Labels/Targets and Process

In [20]:
# predict ddir(t)
chosen_l = pd_split_ternary_to_binary(ld['hoc']['pba']['ddir'].replace(to_replace=-1, value=0))
chosen_t = pd_split_ternary_to_binary(td['hoc']['pba']['dret'])

In [21]:
# # predict thresholded direction: ddir1(t)
# chosen_l_len = len(ld['hoc']['pba']['ddir1_log'].columns)
# chosen_l = pd_split_ternary_to_binary(ld['hoc']['pba']['ddir1_log'])
# chosen_l = chosen_l.sum(level=0)#+len(chosen_l.columns)
# chosen_l = (chosen_l[chosen_l!=0] + chosen_l_len).fillna(0).astype(int)
# chosen_l = df_add_midx_level(chosen_l, 'pba_hoc_hdxret1_ddir1', loc=1, name='id1')

# chosen_t = pd_split_ternary_to_binary(td['hoc']['pba']['dret1_log']).abs().max(level=0)
# chosen_t = df_add_midx_level(chosen_t, 'pba_hoc_hdxret1_dret1', loc=1, name='id1')
# #chosen_l = chosen_l.sum(level=0)#+len(chosen_l.columns)
# #chosen_l

In [22]:
# # 'Predict' the present ddir(t-1), (sanity check)
chosen_l = pd_split_ternary_to_binary(df_del_midx_level(fd['d']['pba']['ddir']['pba_hoc_hdxret_ddir'].rename(columns={-1:'pba_hoc_hdxret_ddir'}), loc=1).replace(to_replace=-1, value=0).astype(int))
chosen_t = pd_split_ternary_to_binary(df_del_midx_level(fd['d']['pba']['dret']['pba_hoc_hdxret_dret'].rename(columns={-1:'pba_hoc_hdxret_dret'}), loc=1))

### Get Common Indexed Rows (Intersect First Level of MultiIndex)

In [23]:
year_interval = ('2009', '2018')
common_idx = midx_intersect(pd_get_midx_level(chosen_f), pd_get_midx_level(chosen_l), pd_get_midx_level(chosen_t))
common_idx = common_idx[(common_idx > year_interval[0]) & (common_idx < year_interval[1])]
feature_df, label_df, target_df = map(compose(partial(pd_rows, idx=common_idx), df_midx_restack), [chosen_f, chosen_l, chosen_t])
assert(all(feature_df.index.levels[0]==label_df.index.levels[0]))
assert(all(feature_df.index.levels[0]==target_df.index.levels[0]))

In [24]:
feature_df

9         10        11  \
id0        id1                                                          
2009-01-02 pba_open_pba_hohlca_hdmx     -0.854887 -1.000000  0.006382   
           pba_high_pba_hohlca_hdmx     -1.000000 -0.330963 -0.113402   
           pba_low_pba_hohlca_hdmx      -1.000000 -0.948160 -0.062400   
           pba_close_pba_hohlca_hdmx    -1.000000 -0.058166  0.020046   
           pba_avgPrice_pba_hohlca_hdmx -1.000000 -0.688267 -0.093641   
...                                           ...       ...       ...   
2017-12-29 pba_open_pba_hohlca_hdmx      1.000000  0.816224  0.422828   
           pba_high_pba_hohlca_hdmx      1.000000  0.475309  0.314815   
           pba_low_pba_hohlca_hdmx       1.000000  0.472727  0.460606   
           pba_close_pba_hohlca_hdmx     1.000000  0.617978  0.702247   
           pba_avgPrice_pba_hohlca_hdmx  1.000000  0.680039  0.451699   

                                               12        13        14  \
id0        id1                                                          
2009-01-02 pba_open_pba_hohlca_hdmx      0.079610  0.261673  0.565334   
           pba_high_pba_hohlca_hdmx      0.058656  0.360114  0.596161   
           pba_low_pba_hohlca_hdmx       0.088000  0.295360  0.470080   
           pba_close_pba_hohlca_hdmx     0.192902  0.493920  0.772593   
           pba_avgPrice_pba_hohlca_hdmx  0.065233  0.360852  0.580325   
...                                           ...       ...       ...   
2017-12-29 pba_open_pba_hohlca_hdmx      0.507538  0.306533  0.007897   
           pba_high_pba_hohlca_hdmx      0.285185  0.124691 -0.077778   
           pba_low_pba_hohlca_hdmx       0.566667  0.175758  0.263636   
           pba_close_pba_hohlca_hdmx     0.507022  0.210674  0.341292   
           pba_avgPrice_pba_hohlca_hdmx  0.539091  0.321021  0.193217   

                                               15        16  
id0        id1                                               
2009-01-02 pba_open_pba_hohlca_hdmx      0.846154  1.000000  
           pba_high_pba_hohlca_hdmx      1.000000  0.791681  
           pba_low_pba_hohlca_hdmx       0.853440  1.000000  
           pba_close_pba_hohlca_hdmx     0.912586  1.000000  
           pba_avgPrice_pba_hohlca_hdmx  0.951078  1.000000  
...                                           ...       ...  
2017-12-29 pba_open_pba_hohlca_hdmx      0.138550 -1.000000  
           pba_high_pba_hohlca_hdmx      0.227160 -1.000000  
           pba_low_pba_hohlca_hdmx      -0.886364 -1.000000  
           pba_close_pba_hohlca_hdmx    -0.786517 -1.000000  
           pba_avgPrice_pba_hohlca_hdmx  0.165438 -1.000000  

[11325 rows x 8 columns]

In [25]:
label_df

/home/kev/miniconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,,neg,pos
id0,id1,,
2009-01-02,pba_hoc_hdxret_ddir,0,1
2009-01-05,pba_hoc_hdxret_ddir,0,0
2009-01-06,pba_hoc_hdxret_ddir,0,1
2009-01-07,pba_hoc_hdxret_ddir,0,0
2009-01-08,pba_hoc_hdxret_ddir,0,1
...,...,...,...
2017-12-22,pba_hoc_hdxret_ddir,0,0
2017-12-26,pba_hoc_hdxret_ddir,0,1
2017-12-27,pba_hoc_hdxret_ddir,0,1


## Debug Data (Runtime Transforms)

### Split into Train/Val/Test and Convert to Numpy Tensor

In [26]:
train_np, val_np, test_np = zip(*map(pd_to_np_tvt, (feature_df, label_df, target_df)))
shapes = np.asarray(tuple(map(lambda tvt: tuple(map(np.shape, tvt)), (train_np, val_np, test_np))))
assert all(np.array_equal(a[:, 1:], b[:, 1:]) for a, b in pairwise(shapes)), 'feature, label, target shapes must be identical across splits'
assert all(len(np.unique(mat.T[0, :]))==1 for mat in shapes), 'first dimension (N) must be identical length in each split for all (feature, label, and target) tensors'

/home/kev/miniconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Runtime Preproc

In [27]:
def __preproc__(data, m_params, overlap=True):
    x, y, z = temporal_preproc_3d(data, window_size=m_params['window_size'], apply_idx=[0]) if (overlap) else stride_preproc_3d(data, window_size=m_params['window_size'])
    if (m_params['loss'] in ('bce', 'bcel', 'ce', 'nll')):
        y_new = np.sum(y, axis=(1, 2), keepdims=False)		# Sum label matrices to scalar values
        if (y.shape[1] > 1):
            y_new += y.shape[1]								# Shift to range [0, C-1]
        if (m_params['loss'] in ('bce', 'bcel')):
            y_new = np.expand_dims(y_new, axis=-1)
        y = y_new
    return (x, y, z)

#@pl.data_loader
def train_dataloader(params, flt):
    logging.info('train_dataloader called')
    return batchify(params, __preproc__(flt, params), False)

#@pl.data_loader
def val_dataloader(params, flt):
    logging.info('val_dataloader called')
    return batchify(params, __preproc__(flt, params), False)

#@pl.data_loader
def test_dataloader(params, flt):
    logging.info('test_dataloader called')
    return batchify(params, __preproc__(flt, params), False)

In [28]:
params = {
    'loss': 'ce',
    'batch_size': 128,
    'window_size': 20,
    'feat_dim': None,
    'feat_shuffle': False
}

### Overlapping Episodes:

In [29]:
train_ol_np = __preproc__(train_np, params)
val_ol_np = __preproc__(val_np, params)
test_ol_np = __preproc__(test_np, params)
print(tuple(map(lambda tvt: tuple(map(np.shape, tvt)), (train_np, val_np, test_np)))[0])
print(tuple(map(np.shape, train_ol_np)))
print(np_value_counts(train_ol_np[1]))

((1359, 5, 8), (1359, 1, 2), (1359, 1, 2))
((1340, 5, 160), (1340,), (1340, 1, 2))
(array([0., 1.]), array([594, 746]))


### Non-Overlapping Episodes:

In [30]:
train_nol_np = __preproc__(train_np, params, False)
val_nol_np = __preproc__(val_np, params, False)
test_nol_np = __preproc__(test_np, params, False)
print(tuple(map(lambda tvt: tuple(map(np.shape, tvt)), (train_np, val_np, test_np)))[0])
print(tuple(map(np.shape, train_nol_np)))

((1359, 5, 8), (1359, 1, 2), (1359, 1, 2))
((67, 5, 20, 8), (67, 2), (67, 1, 20, 2))


## Neural Process

In [31]:
t_params = {
	'window_size': 20,
	'feat_dim': None,
	'train_shuffle': False,    
	'epochs': 50,
	'batch_size': 128,
	'batch_step_size': 64,
	'loss': 'clf',
	'opt': {
		'name': 'adam',
		'kwargs': {
			'lr': .001
		}
	},
	'sch': {
#		'name': 'rpl',
#		'kwargs': {
#			'mode': 'min',
#			'factor': 0.1,
#			'patience': 10,
#			'threshold': 0.0001,
#			'threshold_mode': 'rel',
#			'cooldown': 0,
#			'min_lr': 0
#		}
	},
	'num_workers': 0,
	'pin_memory': True
}

In [32]:
m_params = {
    'out_dist_type': 'normal'
}

In [33]:
%autoreload 1
%aimport model.np_util, model.pl_np, model.pl_generic

In [34]:
from model.np_util import ConvANP
from model.pl_np import NP

/home/kev/miniconda3/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [35]:
train_ol_np[2].shape

(1340, 1, 2)

In [36]:
mdl = NP(ConvANP, m_params, t_params, (feature_df, label_df, target_df))

In [37]:
escb = EarlyStopping(monitor='val_loss', min_delta=0.00, patience=30, verbose=False, mode='min')
trainer = pl.Trainer(max_epochs=t_params['epochs'], auto_lr_find=False, gpus=1, amp_level='O1', precision=16)

GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
TPU available: False, using: 0 TPU cores
INFO:lightning:TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.
INFO:lightning:Using native 16bit precision.


In [38]:
trainer.fit(mdl)

/home/kev/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: RuntimeWarning: You have defined a `val_dataloader()` and have defined a `validation_step()`, you may also want to define `validation_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)
/home/kev/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: RuntimeWarning: You have defined a `test_dataloader()` and have defined a `test_step()`, you may also want to define `test_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)

  | Name  | Type    | Params
----------------------------------
0 | model | ConvANP | 1 M   
INFO:lightning:
  | Name  | Type    | Params
----------------------------------
0 | model | ConvANP | 1 M   
/home/kev/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider in

/home/kev/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: DeprecationWarning: Method `validation_end` was deprecated in v0.7 and will be removed in v1.0. Use `validation_epoch_end` instead.
  warnings.warn(*args, **kwargs)
/home/kev/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/kev/miniconda3/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [39]:
mdl

NP(
  (loss): CrossEntropyLoss()
  (model): ConvANP(
    (det_encoder): DetConvEncoder(
      (target_transform): TemporalConvNet(
        (convnet): Sequential(
          (rb_0): ResidualBlock(
            (net): Sequential(
              (tl_0_0): TemporalLayer1d(
                (layer): Sequential(
                  (0): Conv1d(5, 128, kernel_size=(3,), stride=(1,), padding=(2,))
                  (1): ELU(alpha=1.0)
                  (2): Dropout(p=0.0, inplace=False)
                )
              )
              (tl_0_1): TemporalLayer1d(
                (layer): Sequential(
                  (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(2,))
                  (1): ELU(alpha=1.0)
                  (2): Dropout(p=0.0, inplace=False)
                )
              )
              (tl_0_2): TemporalLayer1d(
                (layer): Sequential(
                  (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(4,))
       

In [ ]:
torch.zeros(20, 10, 5).shape

In [29]:
# deterministic encoder

# latent encoder

In [52]:
train_ol_np[0].shape

(1340, 5, 160)

In [32]:
from model.np_util import DetConvEncoder, LatConvEncoder, ConvDecoder, ConvANP

In [33]:
d_encoder = DetConvEncoder(in_shape=(5, 160), embed_size=128, encoder_depth=3, encoder_kernel_size=3,
                           encoder_dropout=.0, label_size=1, sa_depth=2, sa_heads=8, sa_dropout=0.0,
                           xa_depth=2, xa_heads=8, xa_dropout=0.0)

In [34]:
l_encoder = LatConvEncoder(in_shape=(5, 160), embed_size=128, encoder_depth=3, encoder_kernel_size=3,
                           latent_size=256, label_size=1, sa_depth=2, sa_heads=8, sa_dropout=0.0,
                           min_std=.01, use_lvar=False)

In [35]:
dec = ConvDecoder(in_shape=(5, 160), embed_size=128, decoder_depth=3,
                    decoder_kernel_size=3, decoder_dropout=0.0, label_size=1,
                    encoder_embed_size=128, latent_size=256, min_std=.01, use_lvar=False)

In [36]:
# X_c = torch.from_numpy(train_ol_np[0][0, 0, :])
# y_c = torch.from_numpy(train_ol_np[1][[0]])
# X_t = torch.from_numpy(train_ol_np[0][1, 0, :])
params = {
    'loss': 'ce',
    'batch_size': 128,
    'window_size': 20,
    'feat_dim': None,
    'feat_shuffle': False
}
train_dl = get_dataloader(data=train_np, loss=params['loss'],
                          window_size=params['window_size'], window_overlap=True,
                          feat_dim=params['feat_dim'], batch_size=params['batch_size'],
                          shuffle=params['feat_shuffle'])

In [37]:
for i, (x, y, z) in enumerate(train_dl):
    batch_x = x
    batch_y = y
    batch_z = z
    break

In [37]:
for i, (x, y, z) in enumerate(train_dl):
    if (i==0):
        X_c = x
        y_c = y
    elif (i==1):
        X_t = x
        y_t = y
        break

In [38]:
rep = d_encoder.forward(X_c, y_c, X_t)
print(d_encoder.in_shape, d_encoder.out_shape)
print(rep.shape)

(5, 160) (128, 174)
torch.Size([10, 128, 174])


In [39]:
z_dist, z_logvar = l_encoder.forward(X_c, y_c)
print(l_encoder.in_shape, l_encoder.out_shape)
print(z_dist.rsample().shape)

(5, 160) (256,)
torch.Size([10, 256])


In [40]:
out_dist = dec.forward(rep, z_dist.rsample(), X_t)
print(dec.in_shape, dec.out_shape)
print(out_dist.rsample().shape)

(5, 160) (1,)
torch.Size([10, 1])


In [41]:
cmdl = ConvANP((5, 160))

In [48]:
cmdl

ConvANP(
  (det_encoder): DetConvEncoder(
    (target_transform): TemporalConvNet(
      (convnet): Sequential(
        (rb_0): ResidualBlock(
          (net): Sequential(
            (tl_0_0): TemporalLayer1d(
              (layer): Sequential(
                (0): Conv1d(5, 128, kernel_size=(3,), stride=(1,), padding=(2,))
                (1): ELU(alpha=1.0)
                (2): Dropout(p=0.0, inplace=False)
              )
            )
            (tl_0_1): TemporalLayer1d(
              (layer): Sequential(
                (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(2,))
                (1): ELU(alpha=1.0)
                (2): Dropout(p=0.0, inplace=False)
              )
            )
            (tl_0_2): TemporalLayer1d(
              (layer): Sequential(
                (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(4,))
                (1): ELU(alpha=1.0)
                (2): Dropout(p=0.0, inplace=False)
        

In [43]:
prior_dist, post_dist, out_dist = cmdl.forward(X_c, y_c, X_t, y_t)

In [44]:
training = False
use_lvar = False
context_in_target = False
context_x, context_y, target_x, target_y = X_c, y_c, X_t, y_t

In [45]:
y_pred, loss = cmdl.sample(prior_dist, post_dist, out_dist, target_y)

In [104]:
dmdl = DeterministicEncoder(7, 5, use_self_attn=True)

In [105]:
cx = torch.rand(10, 7, 5)
cy = torch.rand(10,)
tx = torch.rand(10, 7, 5)
dmdl.forward(cx, cy, tx)

RuntimeError: Tensors must have same number of dimensions: got 3 and 1

In [87]:
mdl = LatentEncoder(4+1)

In [85]:
d, lv = mdl.forward(torch.rand(10,5,4), torch.rand(10,5,1))
z = d.rsample()
print(z.shape)

torch.Size([10, 32])


In [86]:
z_unsq = z.unsqueeze(1)
print(z_unsq.shape)
print(z_unsq)
z_ = z_unsq.repeat(1, torch.rand(10,5,4).size(1), 1)
print(z_.shape)
print(z_)

torch.Size([10, 1, 32])
tensor([[[-3.2352e-01, -1.1393e+00, -1.4866e-01, -7.2126e-01, -1.0296e+00,
           1.3941e-01, -5.0069e-03,  3.0221e-02,  3.1430e-01, -6.2683e-01,
          -3.0374e-01,  3.3186e-01, -3.7847e-01, -4.5033e-01, -7.5644e-01,
          -2.5020e-01,  8.4438e-01, -5.2810e-01,  5.3774e-01, -3.2465e-01,
           6.8816e-01, -7.6092e-01,  3.0775e-01,  1.2021e-01, -1.1108e-01,
           1.0471e-01, -6.5850e-01,  7.8725e-01,  4.4444e-01,  4.4447e-02,
          -2.5699e-01, -9.7003e-01]],

        [[ 9.0349e-01, -6.4716e-02, -3.4623e-01,  2.9461e-01,  5.5019e-01,
          -5.3019e-01,  8.5027e-01, -3.5723e-01,  3.5999e-01,  3.7843e-01,
           8.0824e-01,  6.3133e-01, -5.7981e-01,  7.8850e-01, -1.3937e-01,
          -3.7323e-03, -2.6944e-01, -4.4656e-01,  8.4098e-02, -4.6628e-01,
          -2.7654e-01,  2.9140e-01, -2.1933e-01, -8.2534e-01, -9.4757e-02,
          -7.9857e-01, -6.5182e-02, -6.5836e-02, -3.8676e-01,  2.0640e-01,
          -7.9728e-02, -4.3786e-01]],

In [78]:
z = z.unsqueeze(1).repeat(1, .size(1), 1)

NameError: name 'target_x' is not defined

In [52]:
from model.model_util import TemporalConvNet

tcn_ = TemporalConvNet((5, 160),
    block_channels=[[128 for _ in range(3)]],
    pad_type='full', kernel_sizes=[3],
    global_dropout=0.0, no_dropout=None)
x_tcn_out = tcn_.forward(X_t)
print(x_tcn_out.shape)

torch.Size([10, 128, 174])


In [53]:
print('X_t', X_t.shape)
x_ = tcn_.forward(X_t)
print('x_', x_.shape, '\n')

print('rep', rep.shape, 'z_mean', z_mean.unsqueeze(2).shape)
z_ = torch.cat([rep, z_mean.unsqueeze(2)], dim=2)
print('z_', z_.shape, '\n')
      
print('z_', z_.shape, 'x_', x_.shape)
rep = torch.cat([z_, x_], dim=2)
print('rep', rep.shape, '\n')

X_t torch.Size([10, 5, 160])
x_ torch.Size([10, 128, 174]) 

rep torch.Size([10, 128, 350]) z_mean torch.Size([10, 128, 1])
z_ torch.Size([10, 128, 351]) 

z_ torch.Size([10, 128, 351]) x_ torch.Size([10, 128, 174])
rep torch.Size([10, 128, 525]) 



In [54]:
decoder_ = TemporalConvNet((128, 342),
    block_channels=[[128 for _ in range(3)]],
    pad_type='full', kernel_sizes=[3],
    global_dropout=0.0, no_dropout=None)
print('rep', rep.shape)
print(decoder_.forward(rep).shape)

rep torch.Size([10, 128, 525])
torch.Size([10, 128, 539])
